In [2]:
import pandas as pd
import librosa
import os
import numpy as np

In [3]:
def load_data(path_file : str):
    """
    Carga los audios con librosa y los convierte en un dataframe
    """
    data_song = os.path.join(path_file, "Song")
    data_speech = os.path.join(path_file, "Speech")

    labels = []
    data = []
    for actor in os.listdir(data_song):
        actor_path = os.path.join(data_song, actor)
        for file in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file)
            #print(file_path)
            # Load audio file with Librosa
            y, sr = librosa.load(file_path)
            #print(f"Y shape: {y.shape}")

            # Apply MFCC transformation
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
            mfcc = np.mean(mfcc, axis=1)
            #print(f"Mfcc: {mfcc.shape}")
            # Append to data
            data.append(mfcc)
            labels.append(actor_path[-2:])

    for actor in os.listdir(data_speech):
        actor_path = os.path.join(data_speech, actor)
        for file in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file)
            # Load audio file with Librosa
            y, sr = librosa.load(file_path)
            # Apply MFCC transformation 
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
            mfcc = np.mean(mfcc, axis=1)
            # Append to data
            data.append(mfcc)
            labels.append(actor_path[-2:])

    return data, labels

data, labels = load_data("sound_dataset")

In [4]:
data_np = np.array(data)
df = pd.DataFrame(data_np)
for i in range(len(labels)):
    df.loc[i, 'label'] = labels[i]

In [5]:
# Split dataframe into train and test sets
train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)

# Save train and test sets to CSV
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

TypeError: unsupported operand type(s) for -: 'str' and 'str'